In [2]:
import pandas as pd

Trabajo en el siguiente rango de dias:

In [3]:
firstDay = pd.Timestamp(2019, 4, 18)
lastDay = pd.Timestamp(2019, 4, 21)

Los datos son los siguientes

In [4]:
location = 'data/'
auctionsFile = location + 'auctions.csv'
eventsFile = location + 'events.csv'
clicksFile = location + 'clicks.csv'
installsFile = location + 'installs.csv'

Cargo los datos de los primeros 3 dias

In [53]:
dtype = {
    'date': 'str',
    'event_id': 'int8',
    'ref_type': 'category',
    'ref_hash': 'int64',
    'application_id': 'int64',
    'attributed': 'bool',
    'device_os_version': 'float64',
    'device_brand': 'float64',
    'device_model': 'float64',
    'device_city': 'float64',
    'session_user_agent': 'float64',
    'trans_id': 'str',
    'user_agent': 'float64',
    'event_uuid': 'str',
    'carrier': 'float64',
    'kind': 'float64',
    'device_os': 'category',
    'wifi': 'bool',
    'connection_type': 'category',
    'ip_address': 'int64',
    'device_language': 'category'
}

In [ ]:
chunksize = 50000

In [54]:
eventsIterator = pd.read_csv(eventsFile, usecols=dtype.keys(), dtype=dtype, parse_dates=['date'], chunksize=chunksize)

In [55]:
events = pd.DataFrame()
for chunk in eventsIterator:
    chunkIsInDateInterval = (chunk['date'] >= firstDay) & (chunk['date'] < lastDay)
    events = pd.concat([events, chunk[chunkIsInDateInterval]], sort=False)

In [56]:
events.head()

,date,event_id,ref_type,ref_hash,application_id,attributed,device_os_version,device_brand,device_model,device_city,...,trans_id,user_agent,event_uuid,carrier,kind,device_os,wifi,connection_type,ip_address,device_language
0,2019-04-20 01:42:49.120,0,1891515180541284343,5857744372586891366,210,False,NaN,NaN,4.318294e+18,NaN,...,NaN,5.046185e+18,5b506964-5f47-4b28-a8c2-8a92d6c23379,NaN,5.882882e+18,NaN,False,NaN,7544543351571901618,3.3013777759776993e+18
1,2019-04-20 01:42:49.340,1,1891515180541284343,7642521036780133571,210,False,NaN,NaN,NaN,NaN,...,NaN,NaN,f1fb9d15-1a7b-4116-8d3b-c4c403e197e2,NaN,4.017674e+18,NaN,False,NaN,6949523255335024165,NaN
2,2019-04-20 01:42:49.365,1,1891515180541284343,2548841562898283198,210,False,NaN,NaN,NaN,NaN,...,NaN,NaN,c85a0b15-a5d7-472e-8116-6bfa3db19687,NaN,4.017674e+18,NaN,False,NaN,6428537280982666957,NaN
3,2019-04-20 01:42:51.438,2,1891515180541284343,609402887625919085,210,False,NaN,NaN,NaN,NaN,...,NaN,NaN,f4aa0a97-2de6-4f22-95c6-1b3150112cb9,NaN,6.168309e+18,NaN,False,NaN,7607371352198017145,NaN
4,2019-04-20 01:42:51.838,1,1891515180541284343,9114651763556439823,210,False,NaN,NaN,NaN,NaN,...,NaN,NaN,08e2f7f7-875f-4aa0-b337-b9b87b0d83ea,NaN,4.017674e+18,NaN,False,NaN,2901772839007473756,NaN


In [57]:
events.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2261451 entries, 0 to 7744308
Data columns (total 21 columns):
date                  datetime64[ns]
event_id              int8
ref_type              category
ref_hash              int64
application_id        int64
attributed            bool
device_os_version     float64
device_brand          float64
device_model          float64
device_city           float64
session_user_agent    float64
trans_id              object
user_agent            float64
event_uuid            object
carrier               float64
kind                  float64
device_os             category
wifi                  bool
connection_type       category
ip_address            int64
device_language       category
dtypes: bool(2), category(4), datetime64[ns](1), float64(8), int64(3), int8(1), object(2)
memory usage: 276.1+ MB
